In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
coe = pd.DataFrame(columns =['Country', 'Year'])

In [1]:
countries_dict = {"Bolivia": "bolivia", "Brazil Naturals" : "brazil-naturals"  , "Brazil Pulped Naturals":"brazil-pulped-naturals", "Burundi": "burundi-program", "Colombia" : "colombia-third-program" , "Colombia North": "colombia-program", "Colombia South" : "colombia-second-program", "Costa Rica": "costa-rica-program", "El Salvador" : "el-salvador-program", "Guatemala" : "guatemala-program", "Honduras" : "honduras-program" , "Mexico" : "mexico-program", "Nicaragua" : "nicaragua-program", " Peru" : "peru", "Rwanda" : "rwanda-program"}

In [36]:
def get_year_spider(country):
    """
    country: which country to be crawled
    result: list of years that COE performed
    """
    result = []
    url = "https://www.allianceforcoffeeexcellence.org/en/cup-of-excellence/country-programs/" + str(country)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
   
    select = soup.find('select', id = 'id_competition')
    result = [year.text for year in select.find_all('option')]
    return result

In [ ]:
def spider(max_pages):
    page = 1
    while page < max_pages:
        url = "https://www.allianceforcoffeeexcellence.org/en/cup-of-excellence/country-programs/" + str(country) + "/" + str(year)
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'lxml')
        
        
        